In [1]:
# === Updated PyTorch Code with Attention Block, Slice-Level Inference Averaging, and TTA (Fixed CUDA Error) ===

import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from tqdm import tqdm
import random
import re
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models

# Ensure reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Disable cuDNN benchmark to avoid misalignment bugs
torch.backends.cudnn.benchmark = False

# -------- Helper Function -------- #
def get_series_number(img_name):
    match = re.search(r'IM-(\d+)-', img_name)
    return match.group(1) if match else None

# -------- Custom Attention Block -------- #
class AttentionBlock(nn.Module):
    def __init__(self, dim, heads=8):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=heads, batch_first=True)
        self.norm = nn.LayerNorm(dim)
        self.linear = nn.Linear(dim, dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # [B, 1, C]
        attn_output, _ = self.attn(x, x, x)
        x = self.norm(attn_output + x)
        return self.linear(x.squeeze(1))

# -------- Dataset Class -------- #
class FetalMRIDataset(Dataset):
    def __init__(self, root_dir, labels_file, transform=None, tta=False):
        self.root_dir = root_dir
        self.labels_df = pd.read_excel(labels_file)
        self.transform = transform
        self.tta = tta
        self.data = []

        for _, row in self.labels_df.iterrows():
            patient_id = str(row['patient_id'])
            ga_days = row['ga_days']
            ax_series = str(row['ax_series']).zfill(4)
            cor_series = str(row['cor_series']).zfill(4)
            sag_series = str(row['sag_series']).zfill(4)
            folder_path = os.path.join(self.root_dir, patient_id)
            if not os.path.exists(folder_path):
                continue

            views = {'axial': [], 'coronal': [], 'sagittal': []}
            for img_file in os.listdir(folder_path):
                if img_file.endswith(".jpg"):
                    path = os.path.join(folder_path, img_file)
                    series = get_series_number(img_file)
                    if series == ax_series:
                        views['axial'].append(path)
                    elif series == cor_series:
                        views['coronal'].append(path)
                    elif series == sag_series:
                        views['sagittal'].append(path)

            max_len = max(len(v) for v in views.values())
            for axis in views:
                if len(views[axis]) < max_len:
                    views[axis] = random.choices(views[axis], k=max_len)
                else:
                    views[axis] = random.sample(views[axis], k=max_len)

            all_images = views['axial'] + views['coronal'] + views['sagittal']
            for path in all_images:
                self.data.append((path, ga_days))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, label = self.data[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

# -------- Paths -------- #
data_root = "/kaggle/input/fetal-brain/images"
label_path = "/kaggle/input/fetal-brain/images/labels.xlsx"

# -------- Transforms -------- #
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# -------- Dataset -------- #
full_dataset = FetalMRIDataset(data_root, label_path, transform=train_transform)
train_idx, val_idx = train_test_split(np.arange(len(full_dataset)), test_size=0.2, random_state=42)
train_set = torch.utils.data.Subset(full_dataset, train_idx)
val_dataset = FetalMRIDataset(data_root, label_path, transform=val_transform, tta=True)
val_set = torch.utils.data.Subset(val_dataset, val_idx)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, prefetch_factor=4)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=4, pin_memory=True, prefetch_factor=4)

# -------- Model with Attention Block -------- #
class GARegressor(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.efficientnet_b0(pretrained=True)
        self.features = base.features
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        dim = base.classifier[1].in_features
        self.attn = AttentionBlock(dim)
        self.regressor = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(dim, 1)
        )

    def forward(self, x):
        x = self.features(x.contiguous())
        x = self.pool(x)
        x = self.flatten(x)
        x = self.attn(x)
        return self.regressor(x).squeeze(1)

# -------- Training Setup -------- #
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GARegressor()
model = model.to(device)  # Avoid DataParallel due to known CUDA misalignment bugs

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, verbose=True)

best_val_mae = float('inf')
best_val_r2 = float('-inf')
no_improve = 0
patience = 5

# -------- Training Loop -------- #
for epoch in range(1, 51):
    model.train()
    y_true_train, y_pred_train = [], []

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch} - Training"):
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        y_true_train.extend(labels.cpu().numpy())
        y_pred_train.extend(outputs.detach().cpu().numpy())

    train_mae = mean_absolute_error(y_true_train, y_pred_train)
    train_r2 = r2_score(y_true_train, y_pred_train)

    model.eval()
    y_true_val, y_pred_val = [], []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            preds = [model(images).cpu().numpy() for _ in range(3)]
            ensemble_pred = np.mean(preds, axis=0)
            y_true_val.extend(labels.cpu().numpy())
            y_pred_val.extend(ensemble_pred)

    val_mae = mean_absolute_error(y_true_val, y_pred_val)
    val_r2 = r2_score(y_true_val, y_pred_val)
    print(f"📊 Epoch {epoch:02d} | Train MAE: {train_mae:.2f}d, R²: {train_r2:.4f} | Val MAE: {val_mae:.2f}d, R²: {val_r2:.4f}")

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_val_r2 = val_r2
        torch.save(model.state_dict(), "best_model.pth")
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
            print("🛑 Early stopping triggered.")
            break

    scheduler.step(val_mae)

# -------- Final Output -------- #
print(f"\n✅ Best Validation MAE: {best_val_mae:.2f} days | R²: {best_val_r2:.4f}")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 164MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr

📊 Epoch 01 | Train MAE: 35.26d, R²: -1.3331 | Val MAE: 12.91d, R²: 0.7806


Validating: 100%|██████████| 183/183 [00:44<00:00,  4.09it/s]


📊 Epoch 02 | Train MAE: 13.79d, R²: 0.7549 | Val MAE: 11.36d, R²: 0.8346


Validating: 100%|██████████| 183/183 [00:44<00:00,  4.08it/s]


...

📊 Epoch 48 | Train MAE: 4.72d, R²: 0.9736 | Val MAE: 2.58d, R²: 0.9867


Validating: 100%|██████████| 183/183 [00:44<00:00,  4.08it/s]


📊 Epoch 49 | Train MAE: 4.76d, R²: 0.9735 | Val MAE: 2.72d, R²: 0.9859


Validating: 100%|██████████| 183/183 [00:44<00:00,  4.08it/s]

📊 Epoch 50 | Train MAE: 4.70d, R²: 0.9739 | Val MAE: 2.61d, R²: 0.9859

✅ Best Validation MAE: 2.56 days | R²: 0.9864
